In [304]:
import pandas as pd

In [305]:
muni_eda = pd.read_csv('../../Data/Complete_N_Cleaned.csv')
muni_eda.head()

,building_id,district_id,vdcmun_id,ward_id,count_floors_pre_eq,count_floors_post_eq,age_building,plinth_area_sq_ft,height_ft_pre_eq,height_ft_post_eq,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,120101000011,12,1207,120703,1,1,9,288,9,9,...,0,0,0,0,0,0,0,0,0,0
1,120101000021,12,1207,120703,1,1,15,364,9,9,...,0,0,0,0,0,0,0,0,0,0
2,120101000031,12,1207,120703,1,1,20,384,9,9,...,0,0,0,0,0,0,0,0,0,0
3,120101000041,12,1207,120703,1,1,20,312,9,9,...,0,0,0,0,0,0,0,0,0,0
4,120101000051,12,1207,120703,1,1,30,308,9,9,...,0,0,0,0,0,0,0,0,0,0


In [306]:
# need to map the districts to the actual name not just the numerical value
ward_district_mapping = pd.read_csv('../../Data/building_damage_assessment_building_ownership_and_use_building_structure/ward_vdcmun_district_name_mapping.csv')
district_names = dict(zip(ward_district_mapping.district_id, ward_district_mapping.district_name))
muni_names = dict(zip(ward_district_mapping.ward_id, ward_district_mapping.vdcmun_name))
muni_eda['dist_nm'] = muni_eda['district_id'].apply(lambda x: district_names[x])
muni_eda['muni_nm'] = muni_eda['ward_id'].apply(lambda x: muni_names[x])
# formatting details
muni_eda['dist_nm'] = muni_eda['dist_nm'].str.upper()
muni_eda['muni_nm'] = muni_eda['muni_nm'].str.upper()

muni_eda[['district_id','dist_nm','muni_nm','ward_id']].head()

,district_id,dist_nm,muni_nm,ward_id
0,12,OKHALDHUNGA,SIDDHICHARAN MUNICIPALITY,120703
1,12,OKHALDHUNGA,SIDDHICHARAN MUNICIPALITY,120703
2,12,OKHALDHUNGA,SIDDHICHARAN MUNICIPALITY,120703
3,12,OKHALDHUNGA,SIDDHICHARAN MUNICIPALITY,120703
4,12,OKHALDHUNGA,SIDDHICHARAN MUNICIPALITY,120703


In [307]:
# setting indexes for concatenation later
muni_eda.set_index('building_id',inplace=True)
print(muni_eda.index[:4])
muni_eda.head()

Int64Index([120101000011, 120101000021, 120101000031, 120101000041], dtype='int64', name='building_id')


,district_id,vdcmun_id,ward_id,count_floors_pre_eq,count_floors_post_eq,age_building,plinth_area_sq_ft,height_ft_pre_eq,height_ft_post_eq,land_surface_condition,...,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,dist_nm,muni_nm
building_id,,,,,,,,,,,,,,,,,,,,,
120101000011,12,1207,120703,1,1,9,288,9,9,flat,...,0,0,0,0,0,0,0,0,OKHALDHUNGA,SIDDHICHARAN MUNICIPALITY
120101000021,12,1207,120703,1,1,15,364,9,9,flat,...,0,0,0,0,0,0,0,0,OKHALDHUNGA,SIDDHICHARAN MUNICIPALITY
120101000031,12,1207,120703,1,1,20,384,9,9,flat,...,0,0,0,0,0,0,0,0,OKHALDHUNGA,SIDDHICHARAN MUNICIPALITY
120101000041,12,1207,120703,1,1,20,312,9,9,flat,...,0,0,0,0,0,0,0,0,OKHALDHUNGA,SIDDHICHARAN MUNICIPALITY
120101000051,12,1207,120703,1,1,30,308,9,9,flat,...,0,0,0,0,0,0,0,0,OKHALDHUNGA,SIDDHICHARAN MUNICIPALITY


---
Pulling in the shape file to reformat the column names and then export as a geojson.

In [308]:
import geopandas as gpd
path = '../../Data/geo_json_files/local_level_gis/local_level.shp'
geo_df = gpd.read_file(path)
geo_df.head()

,ID,PROVINCE,DISTRICT,UNIT_TYPE,UNIT_NAME,DISTRICT_C,SHAPE_LENG,SHAPE_AREA,geometry
0,1001,1,TAPLEJUNG,Gaunpalika,Aathrai Tribeni,1,44893.148204,8.882693e+07,"POLYGON ((857164.710 3033296.092, 857288.713 3..."
1,1002,1,TAPLEJUNG,Gaunpalika,Maiwakhola,1,61337.336115,1.378230e+08,"POLYGON ((846503.096 3039493.069, 846772.432 3..."
2,1003,1,TAPLEJUNG,Gaunpalika,Meringden,1,76402.938500,2.103250e+08,"POLYGON ((846887.885 3054951.309, 846853.440 3..."
3,1004,1,TAPLEJUNG,Gaunpalika,Mikwakhola,1,123800.756560,4.429564e+08,"POLYGON ((861562.623 3073346.623, 861601.463 3..."
4,1005,1,TAPLEJUNG,Gaunpalika,Phaktanglung,1,229604.803763,1.858506e+09,"POLYGON ((877625.740 3098414.056, 877687.431 3..."


In [309]:
dist_list = list(muni_eda['dist_nm'].unique())
print(dist_list)

['OKHALDHUNGA', 'SINDHULI', 'RAMECHHAP', 'DOLAKHA', 'SINDHUPALCHOK', 'KAVREPALANCHOK', 'NUWAKOT', 'RASUWA', 'DHADING', 'MAKWANPUR', 'GORKHA']


In [310]:
print(sorted(geo_df['DISTRICT'].unique()))

['ACHHAM', 'ARGHAKHANCHI', 'BAGLUNG', 'BAITADI', 'BAJHANG', 'BAJURA', 'BANKE', 'BARA', 'BARDIYA', 'BHAKTAPUR', 'BHOJPUR', 'CHITAWAN', 'DADELDHURA', 'DAILEKH', 'DANG', 'DARCHULA', 'DHADING', 'DHANKUTA', 'DHANUSHA', 'DOLAKHA', 'DOLPA', 'DOTI', 'EASTERN_RUKUM', 'GORKHA', 'GULMI', 'HUMLA', 'ILAM', 'JAJARKOT', 'JHAPA', 'JUMLA', 'KABHREPALANCHOK', 'KAILALI', 'KALIKOT', 'KANCHANPUR', 'KAPILBASTU', 'KASKI', 'KATHMANDU', 'KHOTANG', 'LALITPUR', 'LAMJUNG', 'MAHOTTARI', 'MAKAWANPUR', 'MANANG', 'MORANG', 'MUGU', 'MUSTANG', 'MYAGDI', 'NAWALPUR', 'NUWAKOT', 'OKHALDHUNGA', 'PALPA', 'PANCHTHAR', 'PARASI', 'PARBAT', 'PARSA', 'PYUTHAN', 'RAMECHHAP', 'RASUWA', 'RAUTAHAT', 'ROLPA', 'RUPANDEHI', 'SALYAN', 'SANKHUWASABHA', 'SAPTARI', 'SARLAHI', 'SINDHULI', 'SINDHUPALCHOK', 'SIRAHA', 'SOLUKHUMBU', 'SUNSARI', 'SURKHET', 'SYANGJA', 'TANAHU', 'TAPLEJUNG', 'TERHATHUM', 'UDAYAPUR', 'WESTERN_RUKUM']


In [311]:
# these are the districts mispelled in my dataset
for dist in dist_list:
    if dist not in set(geo_df['DISTRICT'].unique()):
        print(dist)

KAVREPALANCHOK
MAKWANPUR


In [312]:
geo_df[(geo_df['DISTRICT']=='MAKAWANPUR') ].head(2)

,ID,PROVINCE,DISTRICT,UNIT_TYPE,UNIT_NAME,DISTRICT_C,SHAPE_LENG,SHAPE_AREA,geometry
332,31001,3,MAKAWANPUR,Gaunpalika,Bagmati,31,114007.722594,3.117852e+08,"POLYGON ((634654.749 3035157.500, 634696.876 3..."
333,31002,3,MAKAWANPUR,Gaunpalika,Bakaiya,31,132294.520877,3.937544e+08,"POLYGON ((619429.002 3040220.997, 619500.875 3..."


In [313]:
geo_df[(geo_df['DISTRICT']=='KABHREPALANCHOK') ].head(2)

,ID,PROVINCE,DISTRICT,UNIT_TYPE,UNIT_NAME,DISTRICT_C,SHAPE_LENG,SHAPE_AREA,geometry
266,24001,3,KABHREPALANCHOK,Nagarpalika,Banepa,24,45850.579555,5.459736e+07,"POLYGON ((653211.721 3064905.009, 653243.502 3..."
267,24002,3,KABHREPALANCHOK,Gaunpalika,Bethanchowk,24,62496.963149,1.010062e+08,"POLYGON ((649948.291 3049376.043, 650005.108 3..."


In [314]:
# renaming my muni_eda to match the geo data from public record
muni_eda.loc[muni_eda[muni_eda['dist_nm']=='MAKWANPUR'].index, 'dist_nm']='MAKAWANPUR'

In [315]:
muni_eda.loc[muni_eda[muni_eda['dist_nm']=='KAVREPALANCHOK'].index, 'dist_nm'] = 'KABHREPALANCHOK'

In [316]:
dist_list = list(muni_eda['dist_nm'].unique())
# double checking that the districts all match spelling
for dist in dist_list:
    if dist not in set(geo_df['DISTRICT'].unique()):
        print(dist)

Appears that the only difference between the codes here and on the original dataframe *muni_eda* is that there apppears to be an additional 0 between the district values '12' and municipality code '01'. 

However there so appear to be some discrepancies between the codes with a few districts so I will quickly edit those codes before attempting to reformat the *muni_eda* dataframe so that my data does represent the correct location.

In [388]:
# no overwriting needed after inpection
print(dist_list[0])
geo_df[(geo_df['DISTRICT']=='OKHALDHUNGA') ]

OKHALDHUNGA


,ID,PROVINCE,DISTRICT,UNIT_TYPE,UNIT_NAME,DISTRICT_C,SHAPE_LENG,SHAPE_AREA,geometry
112,12001,1,OKHALDHUNGA,Gaunpalika,Champadevi,12,67524.513063,1.269135e+08,"POLYGON ((86.31694 27.37850, 86.31796 27.37817..."
113,12002,1,OKHALDHUNGA,Gaunpalika,Chisankhugadhi,12,59967.873321,1.119305e+08,"POLYGON ((86.57255 27.34139, 86.57318 27.34047..."
114,12003,1,OKHALDHUNGA,Gaunpalika,Khijidemba,12,63964.120360,1.797693e+08,"POLYGON ((86.35959 27.52424, 86.36011 27.52404..."
115,12004,1,OKHALDHUNGA,Gaunpalika,Likhu,12,54519.341217,8.803235e+07,"POLYGON ((86.25943 27.44245, 86.26033 27.44235..."
116,12005,1,OKHALDHUNGA,Gaunpalika,Manebhanjyang,12,59119.757274,1.466087e+08,"POLYGON ((86.50527 27.27724, 86.50648 27.27708..."
117,12006,1,OKHALDHUNGA,Gaunpalika,Molung,12,63784.231196,1.124961e+08,"POLYGON ((86.44870 27.42875, 86.44786 27.42682..."
118,12007,1,OKHALDHUNGA,Nagarpalika,Siddhicharan,12,105816.935073,1.678773e+08,"POLYGON ((86.47486 27.42132, 86.47583 27.42102..."
119,12008,1,OKHALDHUNGA,Gaunpalika,Sunkoshi,12,68645.816160,1.437480e+08,"POLYGON ((86.37716 27.34501, 86.37733 27.34496..."


In [318]:
# no overwriting needed after inpection
print(dist_list[1])
geo_df[(geo_df['DISTRICT']==dist_list[1]) ].head(2)

SINDHULI


,ID,PROVINCE,DISTRICT,UNIT_TYPE,UNIT_NAME,DISTRICT_C,SHAPE_LENG,SHAPE_AREA,geometry
228,20001,3,SINDHULI,Nagarpalika,Dudhouli,20,123408.621650,3.903742e+08,"POLYGON ((724564.823 3003496.152, 724754.397 3..."
229,20002,3,SINDHULI,Gaunpalika,Ghanglekh,20,73762.141524,1.667642e+08,"POLYGON ((671255.062 3033684.364, 671274.717 3..."


In [319]:
# no overwriting needed after inpection
print(dist_list[2])
geo_df[(geo_df['DISTRICT']==dist_list[2]) ].head(2)

RAMECHHAP


,ID,PROVINCE,DISTRICT,UNIT_TYPE,UNIT_NAME,DISTRICT_C,SHAPE_LENG,SHAPE_AREA,geometry
237,21001,3,RAMECHHAP,Gaunpalika,Doramba,21,66609.949426,1.408701e+08,"POLYGON ((685959.375 3056469.958, 686008.947 3..."
238,21002,3,RAMECHHAP,Gaunpalika,Gokulganga,21,102353.557673,1.983926e+08,"POLYGON ((737919.228 3068917.068, 738042.172 3..."


In [320]:
# no overwriting needed after inpection
print(dist_list[3])
geo_df[(geo_df['DISTRICT']==dist_list[3]) ].head(2)

DOLAKHA


,ID,PROVINCE,DISTRICT,UNIT_TYPE,UNIT_NAME,DISTRICT_C,SHAPE_LENG,SHAPE_AREA,geometry
245,22001,3,DOLAKHA,Gaunpalika,Baiteshwor,22,38849.142774,8.049999e+07,"POLYGON ((709752.509 3065176.738, 709770.410 3..."
246,22002,3,DOLAKHA,Nagarpalika,Bhimeshwor,22,59505.437346,1.325070e+08,"POLYGON ((698494.070 3071115.082, 698513.525 3..."


In [321]:
print(dist_list[4])
geo_df[(geo_df['DISTRICT']==dist_list[4])]

SINDHUPALCHOK


,ID,PROVINCE,DISTRICT,UNIT_TYPE,UNIT_NAME,DISTRICT_C,SHAPE_LENG,SHAPE_AREA,geometry
254,23001,3,SINDHUPALCHOK,Gaunpalika,Balefi,23,45921.282351,6.162194e+07,"POLYGON ((676183.769 3081327.502, 676402.820 3..."
255,23002,3,SINDHUPALCHOK,Nagarpalika,Barhabise,23,73337.431602,1.348093e+08,"POLYGON ((680404.270 3090085.398, 680524.063 3..."
256,23003,3,SINDHUPALCHOK,Gaunpalika,Bhotekoshi,23,88473.692075,2.783179e+08,"POLYGON ((688577.951 3104669.248, 688621.981 3..."
257,23004,3,SINDHUPALCHOK,Nagarpalika,Chautara SangachokGadhi,23,69254.226581,1.652520e+08,"POLYGON ((669558.760 3080690.976, 669738.289 3..."
258,23005,3,SINDHUPALCHOK,Gaunpalika,Helambu,23,95863.696063,2.876237e+08,"POLYGON ((650162.577 3116231.942, 650260.585 3..."
259,23006,3,SINDHUPALCHOK,Gaunpalika,Indrawati,23,56642.854054,1.051162e+08,"POLYGON ((665483.636 3082288.789, 665461.577 3..."
260,23007,3,SINDHUPALCHOK,Gaunpalika,Jugal,23,140026.939149,5.965440e+08,"POLYGON ((679792.178 3119707.852, 679877.447 3..."
261,23008,3,SINDHUPALCHOK,Gaunpalika,Lisangkhu Pakhar,23,49105.133346,9.862469e+07,"POLYGON ((685968.635 3069272.022, 685988.183 3..."
262,23009,3,SINDHUPALCHOK,Nagarpalika,Melamchi,23,84009.245445,1.606267e+08,"POLYGON ((658095.677 3084467.153, 658096.715 3..."
263,23010,3,SINDHUPALCHOK,Gaunpalika,Panchpokhari Thangpal,23,117204.906888,4.364408e+08,"POLYGON ((652529.963 3117443.358, 652637.911 3..."


In [322]:
print(muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==2301].index]['muni_nm'][0:1])
print()
print(muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==2302].index]['muni_nm'][0:1])

building_id
231801000191    BAHRABISE MUNICIPALITY
Name: muni_nm, dtype: object

building_id
230601000011    BALEPHI RURAL MUNICIPALITY
Name: muni_nm, dtype: object


In [323]:
# need to overwriting muni_eda 
muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==2301].index, 'vdcmun_id']=23000
muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==2302].index, 'vdcmun_id']=2301
muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==23000].index, 'vdcmun_id']=2302

In [324]:
print(muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==2301].index]['muni_nm'][0:1])
print()
print(muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==2302].index]['muni_nm'][0:1])

building_id
230601000011    BALEPHI RURAL MUNICIPALITY
Name: muni_nm, dtype: object

building_id
231801000191    BAHRABISE MUNICIPALITY
Name: muni_nm, dtype: object


The municipality 'Langtang' needs to be dropped since the gov't provided geo location does not provide the coordinates and there are not other regions avaliable that match. This means that all the indexes that are in the SINDHUPALCHOK district need to be moved up an index value. 

In [325]:
sind_list=[2309,2310,2311,2312,2313]
# nothing is in this municipality so can easily drop this without issues
muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==(sind_list[0]-1)].index, 'vdcmun_id']

Series([], Name: vdcmun_id, dtype: int64)

In [326]:
for val in sind_list:
    muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==val].index, 'vdcmun_id']=(val-1)
muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==2308].index, 'vdcmun_id'].head()

building_id
230101000011    2308
230101000021    2308
230101000031    2308
230101000041    2308
230101000051    2308
Name: vdcmun_id, dtype: int64

In [327]:
# no overwriting needed after inpection
print(dist_list[5])
geo_df[(geo_df['DISTRICT']==dist_list[5]) ].head(2)

KABHREPALANCHOK


,ID,PROVINCE,DISTRICT,UNIT_TYPE,UNIT_NAME,DISTRICT_C,SHAPE_LENG,SHAPE_AREA,geometry
266,24001,3,KABHREPALANCHOK,Nagarpalika,Banepa,24,45850.579555,5.459736e+07,"POLYGON ((653211.721 3064905.009, 653243.502 3..."
267,24002,3,KABHREPALANCHOK,Gaunpalika,Bethanchowk,24,62496.963149,1.010062e+08,"POLYGON ((649948.291 3049376.043, 650005.108 3..."


In [328]:
print(dist_list[6])
geo_df[(geo_df['DISTRICT']==dist_list[6]) ]

NUWAKOT


,ID,PROVINCE,DISTRICT,UNIT_TYPE,UNIT_NAME,DISTRICT_C,SHAPE_LENG,SHAPE_AREA,geometry
300,28001,3,NUWAKOT,Nagarpalika,Belkotgadhi,28,66640.946123,1.555988e+08,"POLYGON ((615315.751 3086212.251, 615417.252 3..."
301,28002,3,NUWAKOT,Nagarpalika,Bidur,28,73126.525390,1.300081e+08,"POLYGON ((610449.561 3096653.991, 610514.190 3..."
302,28003,3,NUWAKOT,Gaunpalika,Dupcheshwar,28,56266.209297,1.316194e+08,"POLYGON ((643396.249 3100591.005, 643452.936 3..."
303,28004,3,NUWAKOT,Gaunpalika,Kakani,28,55498.956930,8.796552e+07,"POLYGON ((618999.252 3084684.001, 619039.374 3..."
304,28005,3,NUWAKOT,Gaunpalika,Kispang,28,51454.486049,8.257260e+07,"POLYGON ((611767.249 3107690.747, 611826.685 3..."
305,28006,3,NUWAKOT,Gaunpalika,Likhu,28,43737.218644,4.788432e+07,"POLYGON ((618202.499 3091119.248, 618223.929 3..."
306,28007,3,NUWAKOT,Gaunpalika,Meghang,28,68681.184477,9.783487e+07,"POLYGON ((606140.810 3105779.995, 606217.688 3..."
307,28008,3,NUWAKOT,Gaunpalika,Panchakanya,28,38365.115904,5.347281e+07,"POLYGON ((633418.436 3090016.247, 633460.751 3..."
308,28009,3,NUWAKOT,Gaunpalika,Shivapuri,28,63354.620479,1.015034e+08,"POLYGON ((632681.632 3086582.498, 632703.190 3..."
309,28010,3,NUWAKOT,Gaunpalika,Suryagadhi,28,45102.108532,4.909311e+07,"POLYGON ((629191.311 3096658.250, 629219.564 3..."


In [329]:
nuwakot_list=[2807,2808,2809,2810,2811,2812,2813]
# nothing is in this municipality so can easily drop this without issues
muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==(nuwakot_list[0]-1)].index, 'vdcmun_id']

Series([], Name: vdcmun_id, dtype: int64)

In [330]:
for val in nuwakot_list:
    muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==val].index, 'vdcmun_id']=(val-1)
muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==(nuwakot_list[0]-1)].index, 'vdcmun_id'].head()

building_id
281301000011    2806
281301000021    2806
281301000031    2806
281301000041    2806
281301000051    2806
Name: vdcmun_id, dtype: int64

In [331]:
print(dist_list[7])
geo_df[(geo_df['DISTRICT']==dist_list[7]) ]

RASUWA


,ID,PROVINCE,DISTRICT,UNIT_TYPE,UNIT_NAME,DISTRICT_C,SHAPE_LENG,SHAPE_AREA,geometry
314,29001,3,RASUWA,Gaunpalika,Gosaikunda,29,209415.339182,9.772836e+08,"POLYGON ((667993.351 3141797.733, 667997.541 3..."
315,29002,3,RASUWA,Gaunpalika,Kalika,29,45965.550339,4.250163e+07,"POLYGON ((621451.373 3100541.745, 621543.938 3..."
316,29003,3,RASUWA,Gaunpalika,Naukunda,29,52431.009984,1.163572e+08,"POLYGON ((631192.872 3106486.748, 631321.997 3..."
317,29004,3,RASUWA,Gaunpalika,Parbati Kunda,29,81977.138265,2.466318e+08,"POLYGON ((620727.940 3134439.499, 620798.438 3..."
318,29005,3,RASUWA,Gaunpalika,Uttargaya,29,73664.920754,1.184494e+08,"POLYGON ((617670.438 3117060.500, 617732.376 3..."


In [332]:
muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==2901].index, 'vdcmun_id']=22222 # aamachhodinmo 
muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==22222].index,['dist_nm','vdcmun_id']]

,dist_nm,vdcmun_id
building_id,,
290301000011,RASUWA,22222
290301000021,RASUWA,22222
290301000031,RASUWA,22222
290301000041,RASUWA,22222
290301000042,RASUWA,22222
...,...,...
290807000731,RASUWA,22222
290807000741,RASUWA,22222
290807000751,RASUWA,22222


In [333]:
muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==2902].index, 'vdcmun_id']=2901  # gosa 
muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==2903].index, 'vdcmun_id']=2902  # kali
muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==2904].index, 'vdcmun_id']=2903  # naukunda

#muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==2905].index, 'vdcmun_id']=333  # uttagaya
#muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==333].index, 'vdcmun_id']=2905  # uttagaya
muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==22222].index, 'vdcmun_id']=2904 # aamachhodinmo 

In [334]:
# matches above so looks good! 
muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==2904].index,['dist_nm','vdcmun_id']].head()

,dist_nm,vdcmun_id
building_id,,
290301000011,RASUWA,2904
290301000021,RASUWA,2904
290301000031,RASUWA,2904
290301000041,RASUWA,2904
290301000042,RASUWA,2904


In [335]:
# no overwriting needed after inpection
print(dist_list[8])
geo_df[(geo_df['DISTRICT']==dist_list[8]) ].head(2)

DHADING


,ID,PROVINCE,DISTRICT,UNIT_TYPE,UNIT_NAME,DISTRICT_C,SHAPE_LENG,SHAPE_AREA,geometry
319,30001,3,DHADING,Gaunpalika,Benighat Rorang,30,99076.191677,2.065245e+08,"POLYGON ((565957.315 3078421.999, 565981.749 3..."
320,30002,3,DHADING,Nagarpalika,Dhunibesi,30,59757.602226,9.864438e+07,"POLYGON ((621416.999 3073294.501, 621442.752 3..."


In [336]:
print(dist_list[9])
geo_df[(geo_df['DISTRICT']==dist_list[9]) ]

MAKAWANPUR


,ID,PROVINCE,DISTRICT,UNIT_TYPE,UNIT_NAME,DISTRICT_C,SHAPE_LENG,SHAPE_AREA,geometry
332,31001,3,MAKAWANPUR,Gaunpalika,Bagmati,31,114007.722594,3.117852e+08,"POLYGON ((634654.749 3035157.500, 634696.876 3..."
333,31002,3,MAKAWANPUR,Gaunpalika,Bakaiya,31,132294.520877,3.937544e+08,"POLYGON ((619429.002 3040220.997, 619500.875 3..."
334,31003,3,MAKAWANPUR,Gaunpalika,Bhimphedi,31,106506.186182,2.452740e+08,"POLYGON ((601814.311 3055301.749, 601819.249 3..."
335,31004,3,MAKAWANPUR,Upamahanagarpalika,Hetauda,31,110278.312595,2.615950e+08,"POLYGON ((596092.502 3045365.751, 596155.438 3..."
336,31005,3,MAKAWANPUR,Gaunpalika,Indrasarowar,31,56757.409894,9.734038e+07,"POLYGON ((620788.374 3060125.250, 620830.813 3..."
337,31006,3,MAKAWANPUR,Gaunpalika,Kailash,31,73438.814568,2.044974e+08,"POLYGON ((590542.813 3064360.997, 590641.624 3..."
338,31007,3,MAKAWANPUR,Gaunpalika,Makawanpurgadhi,31,73887.500329,1.487217e+08,"POLYGON ((614556.501 3044169.249, 614727.499 3..."
339,31008,3,MAKAWANPUR,Gaunpalika,Manahari,31,99740.261173,1.995238e+08,"POLYGON ((573347.122 3054806.746, 573362.186 3..."
340,31009,3,MAKAWANPUR,Gaunpalika,Raksirang,31,95310.616478,2.267006e+08,"POLYGON ((583921.062 3063418.247, 583990.315 3..."
341,31010,3,MAKAWANPUR,Nagarpalika,Thaha,31,81678.736482,1.911194e+08,"POLYGON ((607740.626 3066402.497, 607846.125 3..."


In [337]:
muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==3111].index,['dist_nm','vdcmun_id']].head()

,dist_nm,vdcmun_id
building_id,,
310101000011,MAKAWANPUR,3111
310101000012,MAKAWANPUR,3111
310101000021,MAKAWANPUR,3111
310101000031,MAKAWANPUR,3111
310101001681,MAKAWANPUR,3111


In [338]:
muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==3111].index, 'vdcmun_id']=3188  # gosa 
muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==3110].index, 'vdcmun_id']=3111  # kali
muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==3109].index, 'vdcmun_id']=3110  # naukunda
muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==3188].index,['dist_nm','vdcmun_id']].head()

,dist_nm,vdcmun_id
building_id,,
310101000011,MAKAWANPUR,3188
310101000012,MAKAWANPUR,3188
310101000021,MAKAWANPUR,3188
310101000031,MAKAWANPUR,3188
310101001681,MAKAWANPUR,3188


In [339]:
print(dist_list[10])
geo_df[(geo_df['DISTRICT']==dist_list[10]) ]

GORKHA


,ID,PROVINCE,DISTRICT,UNIT_TYPE,UNIT_NAME,DISTRICT_C,SHAPE_LENG,SHAPE_AREA,geometry
402,36001,4,GORKHA,Gaunpalika,Aarughat,36,78476.716422,1.607943e+08,"POLYGON ((593530.315 3120297.748, 593615.312 3..."
403,36002,4,GORKHA,Gaunpalika,Ajirkot,36,76237.486266,1.980505e+08,"POLYGON ((568620.189 3137879.989, 568700.060 3..."
404,36003,4,GORKHA,Gaunpalika,Bhimsen,36,62367.771496,1.012491e+08,"POLYGON ((567454.375 3104747.499, 567524.124 3..."
405,36004,4,GORKHA,Gaunpalika,Chum Nubri,36,244120.115986,1.648651e+09,"POLYGON ((554908.752 3180516.999, 555121.813 3..."
406,36005,4,GORKHA,Gaunpalika,Dharche,36,141949.717859,6.515155e+08,"POLYGON ((598565.624 3150574.248, 598640.565 3..."
407,36006,4,GORKHA,Gaunpalika,Gandaki,36,54357.753529,1.238640e+08,"POLYGON ((571831.564 3090093.251, 571856.065 3..."
408,36007,4,GORKHA,Nagarpalika,Gorkha,36,80054.032169,1.318647e+08,"POLYGON ((564110.688 3103456.250, 564161.812 3..."
409,36008,4,GORKHA,Nagarpalika,Palungtar,36,65101.112728,1.586190e+08,"POLYGON ((547261.439 3107849.247, 547345.683 3..."
410,36009,4,GORKHA,Gaunpalika,Sahid Lakhan,36,75548.841481,1.489662e+08,"POLYGON ((567251.065 3097397.250, 567289.686 3..."
411,36010,4,GORKHA,Gaunpalika,Siranchok,36,57531.018267,1.216600e+08,"POLYGON ((560220.311 3115250.501, 560271.251 3..."


In [341]:
muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==3603].index, 'vdcmun_id'].head()

building_id
364801000501    3603
364801000731    3603
364801000741    3603
364802000201    3603
364802000211    3603
Name: vdcmun_id, dtype: int64

In [342]:
muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==3603].index, 'vdcmun_id']=3611  # Barpak Sulikot Rural Muni
muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==3611].index, 'vdcmun_id'].head()

building_id
361401000771    3611
361401000011    3611
361401000021    3611
361401000031    3611
361401000041    3611
Name: vdcmun_id, dtype: int64

In [349]:
muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==3604].index, ['vdcmun_id','dist_nm']].sort_values('vdcmun_id').head()

,vdcmun_id,dist_nm
building_id,,
360501000011,3604,GORKHA
364301001651,3604,GORKHA
364301001641,3604,GORKHA
364301001631,3604,GORKHA
364301001621,3604,GORKHA


In [350]:
gorkha_list = [3604,3605,3606,3607,3608,3609,3610,3611]
for val in gorkha_list:
    muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==val].index, 'vdcmun_id']=(val-1)
muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==3611].index, 'vdcmun_id']
muni_eda.loc[muni_eda[muni_eda['dist_nm']=='GORKHA'].index, ['dist_nm','vdcmun_id']].head()
muni_eda.loc[muni_eda[muni_eda['vdcmun_id']==3603].index, ['vdcmun_id','dist_nm']].sort_values('vdcmun_id').head()

,vdcmun_id,dist_nm
building_id,,
360501000011,3603,GORKHA
364301001651,3603,GORKHA
364301001641,3603,GORKHA
364301001631,3603,GORKHA
364301001621,3603,GORKHA


The latest output matches the above cell so checks out!


Now the *muni_eda* is ready to match that of the geo_df then plot! 

In [351]:
# quick func and demo to then apply to the muni_eda dataframe!  
def adding_zero(num):
    string_str=str(num)
    first = string_str[:2]
    middle = '0'
    last = string_str[-2:]
    return int(first + middle +last)
print(type(adding_zero(1207)))
adding_zero(3188)

<class 'int'>


31088

In [352]:
muni_eda['vdcmun_id'].head()

building_id
120101000011    1207
120101000021    1207
120101000031    1207
120101000041    1207
120101000051    1207
Name: vdcmun_id, dtype: int64

In [353]:
# after I do a quality check of the muni codes across both datasets
muni_eda['vdcmun_id'] = muni_eda['vdcmun_id'].apply(adding_zero)
muni_eda['vdcmun_id'].head()

building_id
120101000011    12007
120101000021    12007
120101000031    12007
120101000041    12007
120101000051    12007
Name: vdcmun_id, dtype: int64

Quickly need to export the geo_df to a geo_json and make sure that they match one another:

In [360]:
# exporting as a geojson
# Must be geographic coords, so casting to WGS84.
geo_df = geo_df.to_crs({'init': 'epsg:4326'})

with open('../../Data/geo_json_files/geo_muni.geojson', 'w') as f:
    f.write(geo_df.to_json())

In [419]:
import json
geo_json = json.load(
    open('../../Data/geo_json_files/geo_muni.geojson','r'))

In [420]:
print(geo_json['features'][0].keys())
geo_json['features'][0]['properties']

dict_keys(['id', 'type', 'properties', 'geometry'])


{'DISTRICT': 'TAPLEJUNG',
 'DISTRICT_C': 1,
 'ID': 1001,
 'PROVINCE': 1,
 'SHAPE_AREA': 88826932.5805,
 'SHAPE_LENG': 44893.1482043,
 'UNIT_NAME': 'Aathrai Tribeni',
 'UNIT_TYPE': 'Gaunpalika'}

In [421]:
# this matches what we have in our muni_eda just need map now based on ID! 
geo_json['features'][300]['properties']['ID']

28001

In [422]:
geo_json['features'][300]['id']

'300'

In [423]:
geo_json['features'][300]['properties']

{'DISTRICT': 'NUWAKOT',
 'DISTRICT_C': 28,
 'ID': 28001,
 'PROVINCE': 3,
 'SHAPE_AREA': 155598808.544,
 'SHAPE_LENG': 66640.9461227,
 'UNIT_NAME': 'Belkotgadhi',
 'UNIT_TYPE': 'Nagarpalika'}

In [424]:
# creating dictionary to map muni_eda later
ids = {}
for feature in geo_json['features']:
    #print(feature['id'])
    #print(feature['properties']['DISTRICT'].lower())
    ids[ feature['properties']['ID'] ]  =feature['id']

In [427]:
# looking at what is NOT in the muni listing final spot check
not_in_list = []
cnt=0
for obs in muni_eda['vdcmun_id']:
    if obs not in set(ids.keys()):
        not_in_list.append(obs)
        cnt+=1
print('districts not in both: ',set(not_in_list))
print('num of times in the dataset: ',cnt)
print('differece after dropped: ',muni_eda.shape[0]-cnt)

districts not in both:  {31011}
num of times in the dataset:  5091
differece after dropped:  757015


In [432]:
# drop that one municipality from muni_eda 
indexe2_drop=(muni_eda[muni_eda['vdcmun_id']==31011].index)
muni_eda.drop(indexe2_drop,inplace=True)

In [433]:
# pending formating 
plotable_buildings_muni = muni_eda.reset_index()

plotable_buildings_muni['muni_id']=plotable_buildings_muni['vdcmun_id'].apply(lambda x: ids[x])

In [435]:
plotable_buildings_muni['muni_id'].head()

0    118
1    118
2    118
3    118
4    118
Name: muni_id, dtype: object

In [436]:
# exporting to do viz in a 'muni_eda_viz.ipynb'
plotable_buildings_muni.to_csv('../../Data/geo_json_files/dataframes_for_viz/muni_level_df.csv',index=True)